## PROYECTO 2 ##

**TÍTULO**

Objetivo: 

Identificar regiones de México con mayor potencial de desarrollo económico y social, combinando demografía, nivel educativo y condiciones de vivienda, para optimizar inversiones, servicios o políticas públicas.

In [2]:
import pandas as pd

url1 = 'https://raw.githubusercontent.com/julio-bp/PROYECTO2/refs/heads/main/datos_de_educacion.csv'
df_educacion = pd.read_csv(url1)

In [3]:
url2 = 'https://raw.githubusercontent.com/julio-bp/PROYECTO2/refs/heads/main/datos_poblacion.csv'
df_poblacion = pd.read_csv(url2)

In [4]:
url3 = 'https://raw.githubusercontent.com/julio-bp/PROYECTO2/refs/heads/main/datos_vivienda.csv'
df_vivienda = pd.read_csv(url3)

**Funciones**

In [ ]:
## análisis general

In [5]:
def informe(x):
    print("Información del DataFrame:")
    print(x.info())
    print("Descripción estadística:")
    print(x.describe())
    print("Valores nulos:")
    print(x.isnull().sum())
    print("Tipos de datos:")
    print(x.dtypes)
    print("Primeras 5 filas:")
    print(x.head())
    print("Últimas 5 filas:")
    print(x.tail())
    print("Forma del DataFrame:")
    print(x.shape)
    print("columnas")
    print(x.columns)

In [ ]:
## análisis funciones categóricas

In [ ]:
def anali(x):
    print(x.value_counts())
    print(x.unique())
    print(x.nunique())

In [ ]:
## limpieza 1

In [ ]:
def limpiar(df,column_name):
    df_limpia=df.copy(  )
    df_limpia[column_name] = df_limpia[column_name].astype(str).str.replace('', '', regex=False).str.upper().str.strip()
    
    print(df_limpia[column_name].value_counts())
    print(df_limpia[column_name].nunique())
    return df_limpia

In [ ]:
## duplicados y nulos

In [ ]:
def datos(x):
    print("datos nulos")
    print(x.isnull().sum())
    print("datos totales")
    print(x.count())   
    print("porcentaje de nulos")
    print(x.isnull().sum() / x.shape[0]*100)
    print("datos duplicados")
    print(x.duplicated().sum())

In [ ]:
## para función categorías

In [ ]:
df["Activity"] = df["Activity"].astype(str).str.upper().str.strip()
df["Activity_group"] = "OTHER"

activity_map = {
    "SURF": "SURF",
    "SWIM|BATH|WAD": "SWIM"
}

for palabra, categoria in activity_map.items():
    filtro = df["Activity"].str.contains(palabra, na=False)
    df.loc[filtro, "Activity_group"] = categoria

## ANÁLISIS ##

In [6]:
informe(df_vivienda)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 37 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Unnamed: 0                                                                         33 non-null     int64  
 1   estados                                                                            33 non-null     object 
 2   disponibilidad de bienes: automovil o camioneta                                    33 non-null     float64
 3   disponibilidad de bienes: bicicleta como medio de transporte                       33 non-null     float64
 4   disponibilidad de bienes: lavadora                                                 33 non-null     float64
 5   disponibilidad de bienes: motocicleta o motoneta                                 

In [7]:
informe(df_poblacion)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Columns: 128 entries, Unnamed: 0 to indice de envejecimiento
dtypes: float64(126), int64(1), object(1)
memory usage: 33.1+ KB
None
Descripción estadística:
       Unnamed: 0  densidad de poblacion  edad mediana  edad mediana hombres  \
count    33.00000              33.000000     33.000000             33.000000   
mean     16.00000             302.240228     29.000000             28.212121   
std       9.66954            1062.566832      1.713914              1.709488   
min       0.00000              10.803055     24.000000             24.000000   
25%       8.00000              43.959693     28.000000             27.000000   
50%      16.00000              64.271664     29.000000             28.000000   
75%      24.00000             148.060870     30.000000             29.000000   
max      32.00000            6163.320469     35.000000             34.000000   

       edad mediana mujere

In [8]:
informe(df_educacion)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 22 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   Unnamed: 0                                                                                33 non-null     int64  
 1   estados                                                                                   33 non-null     object 
 2   grado promedio de escolaridad de la poblacion de 15 y mas anos                            33 non-null     float64
 3   poblacion de 6 a 14 anos que no especifico si sabe o no leer y escribir                   33 non-null     float64
 4   poblacion de 6 a 14 anos que no sabe leer y escribir                                      33 non-null     float64
 5   poblacion de 6 a 14 anos que sab

**HIPÓTESIS 1**

Los estados con mayor nivel educativo presentan mejores condiciones generales de vivienda.

Comprobaciones y visualización:

**HIPÓTESIS 2**

Los estados con mayor presión demográfica presentan mayores niveles de hacinamiento en la vivienda.

Comprobaciones y visualización:

**HIPÓTESIS 3**

Los estados con mayor nivel educativo concentran una mayor proporción de población en edad productiva y menor envejecimiento.

Comprobaciones y visualización:

**HIPÓTESIS 4**

Los estados con mayor nivel educativo presentan un mayor acceso a internet en las viviendas.

Comprobaciones y visualización: